In [2]:
!git clone https://github.com/muxspace/facial_expressions.git

fatal: destination path 'facial_expressions' already exists and is not an empty directory.


In [1]:
import csv
# dictionary with key value pair
# key is string name og emotion
# value is array of image names
data = {}

with open('D:/Coding Ninjas Data Science and Machine Learning/Milestone6/12. Facial Emotions Recognition/facial_expressions/data/legend.csv') as f:
    reader = csv.reader(f)
    next(reader)   #skip first row
    for row in reader:
        key = row[2].lower()
        if key in data:
            data[key].append(row[1])
        else:
            data[key] = [row[1]]

In [4]:
### generating directory structure

In [5]:
## image data generator class from keras
# to do image data generator it requires data to be organized in particular format
# master_data that has test and train folders
# in train it should have folders for all categories of data(happy, sad, neutral....)
# in happy folder there should be particular images jpg
# same for test also

In [2]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [7]:
## making directories
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'master_data'

In [10]:
# making sub directory for each emotion in testing and training
for emotion in emotion_list:
    os.mkdir(os.path.join('master_data/training', emotion))
    os.mkdir(os.path.join('master_data/testing', emotion))

In [ ]:
!pip install pytest-shutil

In [11]:
## to fill these directories from images directory in facial_expressions to respective ones
from shutil import copyfile
split_size = 0.8    #80:20 ratio

for emotion, images in data.items():
    # for each emotion split images inside in 8:2 ratio
    train_size = int(split_size*len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]
    
    #D:/Coding Ninjas Data Science and Machine Learning/Milestone6/12. Facial Emotions Recognition/facial_expressions/data/legend.csv
    for image in train_images:
        source = os.path.join('D:/Coding Ninjas Data Science and Machine Learning/Milestone6/12. Facial Emotions Recognition/facial_expressions/images', image)
        dest = os.path.join('D:/Coding Ninjas Data Science and Machine Learning/Milestone6/12. Facial Emotions Recognition/master_data/training', emotion, image)
        copyfile(source, dest)
    for image in test_images:
        source = os.path.join('D:/Coding Ninjas Data Science and Machine Learning/Milestone6/12. Facial Emotions Recognition/facial_expressions/images', image)
        dest = os.path.join('D:/Coding Ninjas Data Science and Machine Learning/Milestone6/12. Facial Emotions Recognition/master_data/testing', emotion, image)
        copyfile(source, dest)

In [3]:
##### ImageDataGenerator class images shld be in array form and lables shld be one hot encoded
### ImageDataGenerator is used for data augmentation rotation, scaling, shifting, fliping
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [8]:
model = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(8, activation='softmax')      # last layer i.e output is 8 cuz it has 8 classes
])
model.compile(optimizer=Adam(lr=0.01), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 10, 10, 64)      

In [15]:
#### train and test model

In [9]:
train_dir = 'D:/Coding Ninjas Data Science and Machine Learning/Milestone6/12. Facial Emotions Recognition/master_data/training'
test_dir = 'D:/Coding Ninjas Data Science and Machine Learning/Milestone6/12. Facial Emotions Recognition/master_data/testing'

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100,100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                    )

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100,100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                    )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [10]:
### early stopping callback
es = EarlyStopping(monitor='val_acc', patience=2, min_delta=0.01)

In [ ]:
model.fit_generator(train_generator,
                    epochs = 10,
                    verbose=1,
                    validation_data=test_generator,
                    )

C:\Users\anush\AppData\Local\Temp\ipykernel_5012\2437270454.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 277s 3s/step - loss: 0.9204 - acc: 0.6070 - val_loss: 1.7070 - val_acc: 0.6116
Epoch 2/10
86/86 [==============================] - 47s 546ms/step - loss: 0.5977 - acc: 0.7950 - val_loss: 1.8929 - val_acc: 0.6572
Epoch 3/10
86/86 [==============================] - 48s 552ms/step - loss: 0.4939 - acc: 0.8279 - val_loss: 1.7804 - val_acc: 0.6678
Epoch 4/10
86/86 [==============================] - 45s 522ms/step - loss: 0.4560 - acc: 0.8447 - val_loss: 1.6921 - val_acc: 0.6692
Epoch 5/10
86/86 [==============================] - 46s 536ms/step - loss: 0.4181 - acc: 0.8549 - val_loss: 1.9297 - val_acc: 0.6765
Epoch 6/10
86/86 [==============================] - 44s 509ms/step - loss: 0.3832 - acc: 0.8639 - val_loss: 2.4231 - val_acc: 0.6740
Epoch 7/10
86/86 [==============================] - 45s 526ms/step - loss: 0.3468 - acc: 0.8797 - val_loss: 2.0801 - val_acc: 0.6659
Epoch 8/10
86/86 [==============================] - 44s 515ms/step - lo